In [1]:
from google.colab import drive
drive.mount("/content/drive/",force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
!pip install pytorch_pretrained_bert
!pip install tools
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from multiprocessing import Pool, cpu_count
from tools import *
import numpy as np
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from torch.utils.data.distributed import DistributedSampler
import logging
from os import listdir
from os.path import isfile, join
import time

logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

start_time = time.time()


     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 645kB 9.0MB/s 
     |████████████████████████████████| 92kB 4.6MB/s 
  Created wheel for tools: filename=tools-0.1.9-cp36-none-any.whl size=46759 sha256=25f6b8aaab4046e3f222e3f783517101fe5e340e4832c30b1bb8f479ded2059a
  Stored in directory: /root/.cache/pip/wheels/87/67/9b/1ca7dcb0b9ebfdc23a00c85a0644abb6fb14f9159a0df8e067
  Created wheel for pytils: filename=pytils-0.3-cp36-none-any.whl size=40357 sha256=c72ce8db0565846b43dce3ff819bceeab01fe64949000c4c85e5167ec1659a98
  Stored in directory: /root/.cache/pip/wheels/d6/f9/dc/4f07d8ee40d9cfca9973b3f4aeff99d0bb69900e5f3dffbf32
Successfully built tools pytils


#Review classification using BERT

In [0]:
!pip install pytorch_pretrained_bert
!pip install tools
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from multiprocessing import Pool, cpu_count
from tools import *
import numpy as np
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from torch.utils.data.distributed import DistributedSampler
import logging
from os import listdir
from os.path import isfile, join
import time

logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

start_time = time.time()

# Data Loading
%cd /content/drive/My Drive/op_spam_training_data
%cd positive_polarity/
%cd deceptive_from_MTurk/

mypath = '/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/fold1'
pd_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]

mypath1 = '/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/fold2'
pd_train_1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

mypath2 = '/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/fold3'
pd_train_2 = [f for f in listdir(mypath2) if isfile(join(mypath2, f))]

mypath3 = '/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/fold4'
pd_train_3 = [f for f in listdir(mypath3) if isfile(join(mypath3, f))]

%cd fold1/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/pd_test.txt', 'w') as outfile:
    for fname in pd_test:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold2/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/pd_train_1.txt', 'w') as outfile:
    for fname in pd_train_1:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold3/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/pd_train_2.txt', 'w') as outfile:
    for fname in pd_train_2:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold4/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/pd_train_3.txt', 'w') as outfile:
    for fname in pd_train_3:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..

import pandas as pd
f = open('pd_test.txt', 'r')
x = f.readlines()
f.close()
pd_test = pd.DataFrame(x)
pd_test = pd_test.rename(columns={0: 'text'})

f = open('pd_train_1.txt', 'r')
x = f.readlines()
f.close()
pd_train_1 = pd.DataFrame(x)
pd_train_1 = pd_train_1.rename(columns={0: 'text'})

f = open('pd_train_2.txt', 'r')
x = f.readlines()
f.close()
pd_train_2 = pd.DataFrame(x)
pd_train_2 = pd_train_2.rename(columns={0: 'text'})

f = open('pd_train_3.txt', 'r')
x = f.readlines()
f.close()
pd_train_3 = pd.DataFrame(x)
pd_train_3 = pd_train_3.rename(columns={0: 'text'})

pd_train_1 = pd_train_1.append(pd_train_2)
pd_train_1 = pd_train_1.append(pd_train_3)
pd_train = pd_train_1
pd_train.to_csv('pd_train.csv')
pd_test.to_csv('pd_test.csv')

%cd ..
%cd truthful_from_TripAdvisor

mypath = '/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/fold1'
pt_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]

mypath1 = '/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/fold2'
pt_train_1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

mypath2 = '/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/fold3'
pt_train_2 = [f for f in listdir(mypath2) if isfile(join(mypath2, f))]

mypath3 = '/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/fold4'
pt_train_3 = [f for f in listdir(mypath3) if isfile(join(mypath3, f))]

%cd fold1/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/pt_test.txt', 'w') as outfile:
    for fname in pt_test:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold2/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/pt_train_1.txt', 'w') as outfile:
    for fname in pt_train_1:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold3/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/pt_train_2.txt', 'w') as outfile:
    for fname in pt_train_2:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold4/
with open('/content/drive/My Drive/op_spam_training_data/positive_polarity/truthful_from_TripAdvisor/pt_train_3.txt', 'w') as outfile:
    for fname in pt_train_3:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..

import pandas as pd
f = open('pt_test.txt', 'r')
x = f.readlines()
f.close()
pt_test = pd.DataFrame(x)
pt_test = pt_test.rename(columns={0: 'text'})

f = open('pt_train_1.txt', 'r')
x = f.readlines()
f.close()
pt_train_1 = pd.DataFrame(x)
pt_train_1 = pt_train_1.rename(columns={0: 'text'})

f = open('pt_train_2.txt', 'r')
x = f.readlines()
f.close()
pt_train_2 = pd.DataFrame(x)
pt_train_2 = pt_train_2.rename(columns={0: 'text'})

f = open('pt_train_3.txt', 'r')
x = f.readlines()
f.close()
pt_train_3 = pd.DataFrame(x)
pt_train_3 = pt_train_3.rename(columns={0: 'text'})

pt_train_1 = pt_train_1.append(pt_train_2)
pt_train_1 = pt_train_1.append(pt_train_3)
pt_train = pt_train_1
pt_train.to_csv('pt_train.csv')
pt_test.to_csv('pt_test.csv')

%cd ..
%cd ..
%cd negative_polarity/
%cd deceptive_from_MTurk/

from os import listdir
from os.path import isfile, join

mypath = '/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/fold1'
nd_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]

mypath1 = '/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/fold2'
nd_train_1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

mypath2 = '/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/fold3'
nd_train_2 = [f for f in listdir(mypath2) if isfile(join(mypath2, f))]

mypath3 = '/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/fold4'
nd_train_3 = [f for f in listdir(mypath3) if isfile(join(mypath3, f))]

%cd fold1/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/nd_test.txt', 'w') as outfile:
    for fname in nd_test:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold2/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/nd_train_1.txt', 'w') as outfile:
    for fname in nd_train_1:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold3/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/nd_train_2.txt', 'w') as outfile:
    for fname in nd_train_2:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold4/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/deceptive_from_MTurk/nd_train_3.txt', 'w') as outfile:
    for fname in nd_train_3:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..

import pandas as pd
f = open('nd_test.txt', 'r')
x = f.readlines()
f.close()
nd_test = pd.DataFrame(x)
nd_test = nd_test.rename(columns={0: 'text'})

f = open('nd_train_1.txt', 'r')
x = f.readlines()
f.close()
nd_train_1 = pd.DataFrame(x)
nd_train_1 = nd_train_1.rename(columns={0: 'text'})

f = open('nd_train_2.txt', 'r')
x = f.readlines()
f.close()
nd_train_2 = pd.DataFrame(x)
nd_train_2 = nd_train_2.rename(columns={0: 'text'})

f = open('nd_train_3.txt', 'r')
x = f.readlines()
f.close()
nd_train_3 = pd.DataFrame(x)
nd_train_3 = nd_train_3.rename(columns={0: 'text'})

nd_train_1 = nd_train_1.append(nd_train_2)
nd_train_1 = nd_train_1.append(nd_train_3)
nd_train = nd_train_1
nd_train.to_csv('nd_train.csv')
nd_test.to_csv('nd_test.csv')

%cd ..
%cd truthful_from_Web

mypath = '/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/fold1'
nt_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]

mypath1 = '/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/fold2'
nt_train_1 = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]

mypath2 = '/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/fold3'
nt_train_2 = [f for f in listdir(mypath2) if isfile(join(mypath2, f))]

mypath3 = '/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/fold4'
nt_train_3 = [f for f in listdir(mypath3) if isfile(join(mypath3, f))]

%cd fold1/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/nt_test.txt', 'w') as outfile:
    for fname in nt_test:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold2/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/nt_train_1.txt', 'w') as outfile:
    for fname in nt_train_1:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold3/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/nt_train_2.txt', 'w') as outfile:
    for fname in nt_train_2:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..
%cd fold4/
with open('/content/drive/My Drive/op_spam_training_data/negative_polarity/truthful_from_Web/nt_train_3.txt', 'w') as outfile:
    for fname in nt_train_3:
        with open(fname) as infile:
            outfile.write(infile.read())
%cd ..

import pandas as pd
f = open('nt_test.txt', 'r')
x = f.readlines()
f.close()
nt_test = pd.DataFrame(x)
nt_test = nt_test.rename(columns={0: 'text'})

f = open('nt_train_1.txt', 'r')
x = f.readlines()
f.close()
nt_train_1 = pd.DataFrame(x)
nt_train_1 = nt_train_1.rename(columns={0: 'text'})

f = open('nt_train_2.txt', 'r')
x = f.readlines()
f.close()
nt_train_2 = pd.DataFrame(x)
nt_train_2 = nt_train_2.rename(columns={0: 'text'})

f = open('nt_train_3.txt', 'r')
x = f.readlines()
f.close()
nt_train_3 = pd.DataFrame(x)
nt_train_3 = nt_train_3.rename(columns={0: 'text'})

nt_train_1 = nt_train_1.append(nt_train_2)
nt_train_1 = nt_train_1.append(nt_train_3)
nt_train = nt_train_1
nt_train.to_csv('nt_train.csv')
nt_test.to_csv('nt_test.csv')

%cd ..
%cd ..

#creating training and testing data
p_train = pd_train.append(pt_train)
p_test = pd_test.append(pt_test)
n_train = nd_train.append(nt_train)
n_test = nd_test.append(nt_test)

#adding labels to training and testing data
pos_label = []
neg_label = []
for i in range(480):
  pos_label.append(1)
  neg_label.append(0)
p_train['label'] = pos_label
n_train['label'] = neg_label
train_data = p_train.append(n_train)

pos_label = []
neg_label = []
for i in range(160):
  pos_label.append(1)
  neg_label.append(0)
p_test['label'] = pos_label
n_test['label'] = neg_label
test_data = p_test.append(n_test)

#formatting training and testing data as BERT takes input
train_set = pd.DataFrame({'id':range(len(train_data)),'label':train_data['label'],'text': train_data['text'].replace(r'\n', ' ', regex=True)})
alpha = []
for i in range(len(train_data)):
  alpha.append('a')

train_set.insert(2,'alpha',alpha,allow_duplicates=True)
test_set = pd.DataFrame({'id':range(len(test_data)),'label':test_data['label'],'text': test_data['text'].replace(r'\n', ' ', regex=True)})

alpha = []
for i in range(len(test_data)):
  alpha.append('a')

test_set.insert(2,'alpha',alpha,allow_duplicates=True)

train_set.to_csv('train_set.tsv', sep='\t', index=False, header=False)
test_set.to_csv('test_set.tsv', sep='\t', index=False, header=False)




     |████████████████████████████████| 133kB 5.0MB/s 
     |████████████████████████████████| 645kB 54.1MB/s 
     |████████████████████████████████| 92kB 6.9MB/s 
  Created wheel for tools: filename=tools-0.1.9-cp36-none-any.whl size=46759 sha256=ecd7d34a0f2d25d4ab79e15c10860442c90a23c549f8a82c07366ed561b2145e
  Stored in directory: /root/.cache/pip/wheels/87/67/9b/1ca7dcb0b9ebfdc23a00c85a0644abb6fb14f9159a0df8e067
  Created wheel for pytils: filename=pytils-0.3-cp36-none-any.whl size=40357 sha256=004109cdb7c87590f2322d55ad2f517a9748f31d10f9297e5e095fcccc3df100
  Stored in directory: /root/.cache/pip/wheels/d6/f9/dc/4f07d8ee40d9cfca9973b3f4aeff99d0bb69900e5f3dffbf32
Successfully built tools pytils
/content/drive/My Drive/op_spam_training_data
/content/drive/My Drive/op_spam_training_data/positive_polarity
/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk
/content/drive/My Drive/op_spam_training_data/positive_polarity/deceptive_from_MTurk/fold1
/cont

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmpf_o7o0qa
100%|██████████| 213450/213450 [00:00<00:00, 1076570.88B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpf_o7o0qa to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpf_o7o0qa
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to /tmp/tmpgy5dkv5y
100%|██████████| 404400730/404400730 [00:11<00:00, 34128088.18B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpgy5dkv5y to cache at /content/drive/My Drive/op_spam_training_data/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /content/drive/My Drive/op_spam_training_data/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpgy5dkv5y
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /content/drive/My Drive/op_spam_tr

0.266766

Epoch:  20%|██        | 1/5 [00:18<01:12, 18.15s/it]

...

0.003916



INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file /content/drive/My Drive/op_spam_training_data/reviews/vocab.txt


INFO:pytorch_pretrained_bert.modeling:loading archive file /content/drive/My Drive/op_spam_training_data/reviews/
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



INFO:root:***** Eval results *****
INFO:root:  task = reviews
INFO:root:  mcc = 0.8875693440638632
INFO:root:  tp = 152
INFO:root:  tn = 150
INFO:root:  fp = 10
INFO:root:  fn = 8
INFO:root:  Accuracy = 0.94375
INFO:root:  precision = 0.9382716049382716
INFO:root:  recall = 0.95
INFO:root:  f1 score = 0.9440993788819876
INFO:root:  eval_loss = 0.2574174812063575



total run time: 8.872048000494638 minutes


In [0]:
#BERT implementation section
from __future__ import absolute_import, division, print_function

import csv
import os
import sys
import logging

csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.
logger = logging.getLogger()

class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, label=None):

        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):

    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):

    def get_train_examples(self, data_dir):

        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train_set.tsv")), "train")

    def get_dev_examples(self, data_dir):

        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test_set.tsv")), "test")

    def get_labels(self):

        return ["0", "1"]

    def _create_examples(self, lines, set_type):

        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

class InputFeatures(object):

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):

    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_example_to_feature(example_row):

    example, label_map, max_seq_length, tokenizer, output_mode = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)

    if tokens_b:
        tokens += tokens_b + ["[SEP]"]
        segment_ids += [1] * (len(tokens_b) + 1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_mask = [1] * len(input_ids)

    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                         input_mask=input_mask,
                         segment_ids=segment_ids,
                         label_id=label_id)


In [7]:
data_directory = "/content/drive/My Drive/bert"

#initial baseline bert model
base_bert_model = 'bert-base-cased'

sub_folder_name = 'reviews'

output_directory = f'/content/drive/My Drive/bert/{sub_folder_name}/'

evaluation_directory = f'/content/drive/My Drive/bert/{sub_folder_name}_evaluation_report/'

cache_directory = '/content/drive/My Drive/bert/cache/'

#hyperparameters
sequence_length = 128
batch_size = 24
evaluation_batch_size = 32
learn_rate = 2e-5
epochs = 5
seed_val = 42
step_size = 1
initial_prop = 0.1
out_type = 'classification'
config_file = "bert_config.json"
model_file = "pytorch_model.bin"   

if os.path.exists(evaluation_directory) and os.listdir(evaluation_directory):
        evaluation_directory += f'/report_{len(os.listdir(evaluation_directory))}'
        os.makedirs(evaluation_directory)
if not os.path.exists(evaluation_directory):
    os.makedirs(evaluation_directory)
    evaluation_directory += f'/report_{len(os.listdir(evaluation_directory))}'
    os.makedirs(evaluation_directory)

if os.path.exists(output_directory) and os.listdir(output_directory):
        raise ValueError("Please delete existing output directory".format(output_directory))
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

classifier = BinaryClassificationProcessor()
training_data = classifier.get_train_examples(data_directory)
training_data_size = len(training_data)
labels = classifier.get_labels()
num_labels = len(labels)    

train_steps = int(training_data_size / batch_size / step_size) * epochs

tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

labelling = {label: i for i, label in enumerate(labels)}
train_examples_for_processing = [(data, labelling, sequence_length, tokenizer, out_type) for data in training_data]

if __name__ ==  '__main__':
    with Pool(cpu_count() - 1) as p:
        train_features = list(tqdm_notebook(p.imap(convert_example_to_feature, train_examples_for_processing), total=training_data_size))

with open(data_directory + "train_pickle.pkl", "wb") as f:
    pickle.dump(train_features, f)

model = BertForSequenceClassification.from_pretrained(base_bert_model, cache_dir=cache_directory, num_labels=num_labels)
model.to(device)

tune_parameters = list(model.named_parameters())
decay_lst = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in tune_parameters if not any(i in n for i in decay_lst)], 'weight_decay': 0.01},
    {'params': [p for n, p in tune_parameters if any(i in n for i in decay_lst)], 'weight_decay': 0.0}
    ]

optimizer = BertAdam(optimizer_grouped_parameters,lr=learn_rate,warmup=initial_prop,t_total=train_steps)

all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)



INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmpws8o4ce2
100%|██████████| 213450/213450 [00:00<00:00, 5176012.56B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpws8o4ce2 to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpws8o4ce2
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /content/drive/My Drive/bert/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file /content/drive/My Drive/bert/cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /tmp/tmp0jje7pa2
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:pytorch_pretrained_bert.modeling:Weights of BertForSequenceCl

In [0]:
#hyperparameters
sequence_length = 128
batch_size = 24
evaluation_batch_size = 32
learn_rate = 2e-5
epochs = 1
seed_val = 42
step_size = 1
initial_prop = 0.1
out_type = 'classification'
config_file = "bert_config.json"
model_file = "pytorch_model.bin" 

In [9]:
#training
global_step = 0
nb_tr_steps = 0
tr_loss = 0
model.train()
for _ in trange(int(epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

        
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
               
        if step_size > 1:
            loss = loss / step_size

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % step_size == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1


#picks up trained model
model_directory = "/content/drive/My Drive/bert/reviews/"
model_to_save = model.module if hasattr(model, 'module') else model

output_model_file = os.path.join(model_directory, model_file)
output_config_file = os.path.join(model_directory, config_file)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(model_directory)

if os.path.exists(evaluation_directory) and os.listdir(evaluation_directory):
        evaluation_directory += f'/report_{len(os.listdir(evaluation_directory))}'
        os.makedirs(evaluation_directory)
if not os.path.exists(evaluation_directory):
    os.makedirs(evaluation_directory)
    evaluation_directory += f'/report_{len(os.listdir(evaluation_directory))}'
    os.makedirs(evaluation_directory)

#evaluation function
def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    Accuracy = float(tn + tp)/float(tn+tp+fn+fp)
    Precision = float(tp/(tp+fp))
    Recall = float(tp/(tp+fn))
    F1_score = 2*float(Recall * Precision) / float(Recall + Precision)
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "Accuracy": Accuracy,
        "precision": Precision,
        "recall": Recall,
        "f1 score": F1_score
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

tokenizer = BertTokenizer.from_pretrained(model_directory + 'vocab.txt', do_lower_case=False)   

processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(data_directory)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, sequence_length, tokenizer, out_type) for example in eval_examples]

process_count = cpu_count() - 1
if __name__ ==  '__main__':
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long) 

all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=evaluation_batch_size)

#uses the trained model now
model = BertForSequenceClassification.from_pretrained(model_directory, cache_dir=cache_directory, num_labels=len(label_list))

model.to(device)

eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    loss_fct = CrossEntropyLoss()
    tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    
    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
preds = np.argmax(preds, axis=1)

result = compute_metrics(sub_folder_name, all_label_ids.numpy(), preds)

result['eval_loss'] = eval_loss

output_eval_file = os.path.join(evaluation_directory, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in (result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

end_time = time.time()
total_run_time = (end_time - start_time)/60
print('total run time:', total_run_time,'minutes')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

0.008581

Epoch: 100%|██████████| 1/1 [1:33:21<00:00, 5601.13s/it]

0.549300



INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file /content/drive/My Drive/bert/reviews/vocab.txt


INFO:pytorch_pretrained_bert.modeling:loading archive file /content/drive/My Drive/bert/reviews/
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



INFO:root:***** Eval results *****
INFO:root:  task = reviews
INFO:root:  mcc = 0.6734771867817623
INFO:root:  tp = 25692
INFO:root:  tn = 1039
INFO:root:  fp = 843
INFO:root:  fn = 165
INFO:root:  Accuracy = 0.9636612711345037
INFO:root:  precision = 0.9682306387789712
INFO:root:  recall = 0.9936187492748578
INFO:root:  f1 score = 0.9807604214383875
INFO:root:  eval_loss = 0.09810592391610902



total run time: 144.0108384569486 minutes
